# 🔬 ISIC Cilt Lezyonu Analizi - Sayısal Görüntü İşleme

**Proje Sahibi:** Mustafa Engin Dalgıç  
**Öğrenci No:** 254309502  
**Üniversite:** Üsküdar Üniversitesi - Bilgisayar Mühendisliği YL  
**Email:** engindalgic86@gmail.com

---

## ÇALIŞMA YÖNTEMİ

### TEK ADIM:
**`Runtime > Run all`** (veya `Ctrl+F9`)

⏱️ **İşlem Süresi:** 10-15 dakika ( Demo için 1-2 dk) 
📊 **Çıktı:** ~45 grafik

---

In [ ]:
# ==================== BAŞLANGIÇ ====================
print("="*70)
print("🔬 ISIC CİLT LEZYONU ANALİZİ")
print("="*70)
print("\n📦 Kütüphaneler yükleniyor...\n")

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# OpenCV kurulumu
try:
    import cv2
except:
    print("⚠️  OpenCV kuruluyor...")
    !pip install opencv-python -q
    import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import urllib.request
import zipfile
import shutil

# Matplotlib ayarları
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (12, 8)

print(f"✅ OpenCV {cv2.__version__}")
print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}")
print(f"✅ Matplotlib hazır")
print("\n✅ Tüm kütüphaneler başarıyla yüklendi!\n")

In [ ]:
# ==================== PROJE İNDİRME ====================
print("="*70)
print("📥 GITHUB DEPOSUNDAN PROJE İNDİRİLİYOR")
print("="*70 + "\n")

# Ana dizinde çalış
os.chdir("/content")

# Temizlik
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS"):
    shutil.rmtree("UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main"):
    shutil.rmtree("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main")
if os.path.exists("project.zip"):
    os.remove("project.zip")

print(f"📂 Çalışma dizini: {os.getcwd()}")

# GitHub'dan ZIP olarak indir
repo_url = "https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/archive/refs/heads/main.zip"

print("\n📦 ZIP dosyası indiriliyor...")
urllib.request.urlretrieve(repo_url, "project.zip")
print("✅ İndirme tamamlandı")

print("📂 ZIP açılıyor...")
with zipfile.ZipFile("project.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
print("✅ ZIP açıldı")

# Klasör kontrolü ve yeniden adlandırma
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main"):
    print("📝 Klasör adı düzeltiliyor...")
    os.rename("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main", "UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
    print("✅ Klasör adı düzeltildi")

# Proje klasörüne geç
project_dir = "/content/UU_DIP_ISIC_SKIN_LESION_ANALYSIS"
os.chdir(project_dir)
print(f"\n✅ Proje klasörüne geçildi: {os.getcwd()}")

# İçeriği kontrol et
print("\n📋 Proje dosyaları:")
files = os.listdir(".")
for f in sorted(files)[:10]:
    if os.path.isdir(f):
        print(f"   📁 {f}/")
    else:
        print(f"   📄 {f}")
if len(files) > 10:
    print(f"   ... ve {len(files)-10} tane daha")

print("\n" + "="*70)
print("✅ PROJE BAŞARIYLA İNDİRİLDİ")
print("="*70 + "\n")

In [ ]:
# ==================== ANA PROGRAMI ÇALIŞTIR ====================
print("="*70)
print("🚀 ANA PROGRAM BAŞLIYOR")
print("="*70)
print("\n⏱️  Tahmini süre: 10-15 dakika")
print("📊 ~45 grafik oluşturulacak")
print(f"📂 Çalışma dizini: {os.getcwd()}")
print("\nLütfen bekleyin...\n")
print("="*70 + "\n")

# isic_project.py dosyasını çalıştır
!python isic_project.py

print("\n" + "="*70)
print("🎉 PROGRAM BAŞARIYLA TAMAMLANDI!")
print("="*70)

---

## 📊 Sonuçlar

Program başarıyla tamamlandı! Oluşturulan grafikler yukarıda görüntülendi.

### ✅ Uygulanan Teknikler:

1. **Veri Analizi** - Görüntü özellikleri ve dağılımları
2. **RGB vs Grayscale** - Renk uzayı karşılaştırmaları
3. **Histogram Analizi** - Piksel değer dağılımları
4. **Kontrast İyileştirme** - CLAHE, Gamma Correction
5. **Gürültü Azaltma** - Median, Gaussian, Bilateral filtreleme
6. **Geometrik Dönüşümler** - Döndürme, ayna, perspektif
7. **Frekans Filtreleme** - FFT, keskinleştirme

### 📈 Çıktılar:
- Toplam ~45 grafik
- 7 ana bölüm
- 15+ farklı görüntü işleme tekniği

---

## 📁 Dosyaları İndirme (Opsiyonel)

Tüm grafikleri ZIP olarak indirmek için:

In [ ]:
# ==================== GRAFİKLERİ ZIP'LE ====================
print("📦 Grafikler ZIP'leniyor...\n")

import glob

# Proje dizininde olduğumuzu kontrol et
os.chdir("/content/UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
print(f"📂 Konum: {os.getcwd()}\n")

# PNG dosyalarını bul
png_files = glob.glob("*.png")

if len(png_files) > 0:
    # results klasörü oluştur
    if not os.path.exists("results"):
        os.makedirs("results")
    
    # PNG'leri results klasörüne kopyala
    for f in png_files:
        shutil.copy(f, "results/")
    
    # ZIP oluştur
    shutil.make_archive("isic_grafikleri", 'zip', "results")
    
    print(f"✅ {len(png_files)} grafik ZIP'lendi")
    print(f"\n📊 Oluşturulan grafikler:")
    for i, f in enumerate(sorted(png_files)[:10], 1):
        print(f"   {i}. {f}")
    if len(png_files) > 10:
        print(f"   ... ve {len(png_files)-10} tane daha")
    
    print("\n📥 İndirmek için:")
    print("   Sol menüden 📁 Files > UU_DIP_ISIC_SKIN_LESION_ANALYSIS > isic_grafikleri.zip")
    print("   Sağ tık > Download")
else:
    print("⚠️  Grafik bulunamadı!")
    print(f"\n📂 Mevcut dosyalar:")
    for f in os.listdir(".")[:20]:
        print(f"   - {f}")

---

## 💬 İletişim

**Mustafa Engin Dalgıç**  
📧 engindalgic86@gmail.com  
🎓 Üsküdar Üniversitesi - Bilgisayar Mühendisliği Yüksek Lisans  
🔗 [GitHub Repository](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)

---

## 📚 Kaynaklar

- [ISIC Archive](https://www.isic-archive.com/)
- [OpenCV Documentation](https://docs.opencv.org/)
- [Kaggle ISIC Dataset](https://www.kaggle.com/datasets/nodoubttome/skin-cancer9-classesisic)

---

⭐ **Projeyi beğendiyseniz GitHub'da yıldız vermeyi unutmayın!** ⭐

---

<div align="center">
  <img src="https://img.shields.io/badge/Python-3.8+-blue.svg" alt="Python">
  <img src="https://img.shields.io/badge/OpenCV-4.x-green.svg" alt="OpenCV">
  <img src="https://img.shields.io/badge/Status-Complete-success.svg" alt="Status">
</div>